In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [136]:
df = pd.read_csv("BRCA Data.csv")
df.head(3)

,Patient_ID,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,TCGA-D8-A1XD,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Modified Radical Mastectomy,15-Jan-17,19-Jun-17,Alive
1,TCGA-EW-A1OX,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Positive,Positive,Negative,Lumpectomy,26-Apr-17,09-Nov-18,Dead
2,TCGA-A8-A079,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,08-Sep-17,09-Jun-18,Alive


In [137]:
df.shape

(341, 16)

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Patient_ID          334 non-null    object 
 1   Age                 334 non-null    float64
 2   Gender              334 non-null    object 
 3   Protein1            334 non-null    float64
 4   Protein2            334 non-null    float64
 5   Protein3            334 non-null    float64
 6   Protein4            334 non-null    float64
 7   Tumour_Stage        334 non-null    object 
 8   Histology           334 non-null    object 
 9   ER status           334 non-null    object 
 10  PR status           334 non-null    object 
 11  HER2 status         334 non-null    object 
 12  Surgery_type        334 non-null    object 
 13  Date_of_Surgery     334 non-null    object 
 14  Date_of_Last_Visit  317 non-null    object 
 15  Patient_Status      321 non-null    object 
dtypes: float

In [139]:
df.isnull().sum()

Patient_ID             7
Age                    7
Gender                 7
Protein1               7
Protein2               7
Protein3               7
Protein4               7
Tumour_Stage           7
Histology              7
ER status              7
PR status              7
HER2 status            7
Surgery_type           7
Date_of_Surgery        7
Date_of_Last_Visit    24
Patient_Status        20
dtype: int64

In [140]:
df.dropna(inplace=True)

In [141]:
df.isnull().sum()

Patient_ID            0
Age                   0
Gender                0
Protein1              0
Protein2              0
Protein3              0
Protein4              0
Tumour_Stage          0
Histology             0
ER status             0
PR status             0
HER2 status           0
Surgery_type          0
Date_of_Surgery       0
Date_of_Last_Visit    0
Patient_Status        0
dtype: int64

In [142]:
import plotly.express as px

In [143]:
for i in df.iloc[:,2:]:
    status = df[i].value_counts()
    fig = px.pie(values= status.values, names= status.index, hole= 0.5,title= i, width= 700,height= 400)
    fig.show()
    

In [144]:
df.columns

Index(['Patient_ID', 'Age', 'Gender', 'Protein1', 'Protein2', 'Protein3',
       'Protein4', 'Tumour_Stage', 'Histology', 'ER status', 'PR status',
       'HER2 status', 'Surgery_type', 'Date_of_Surgery', 'Date_of_Last_Visit',
       'Patient_Status'],
      dtype='object')

In [145]:
df.drop(['Patient_ID','ER status', 'PR status', 'Date_of_Surgery', 'Date_of_Last_Visit'],axis=1,inplace= True)
df.head(3)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,Patient_Status
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Negative,Modified Radical Mastectomy,Alive
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Negative,Lumpectomy,Dead
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Negative,Other,Alive


In [146]:
encode = pd.get_dummies(df[['Gender','Tumour_Stage','Histology','HER2 status','Surgery_type','Patient_Status']],drop_first=True).astype(int)
encode.head(3)

,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,0,0,1,0,0,0,1,0,0,0
1,0,1,0,0,1,0,0,0,0,1
2,0,0,1,0,0,0,0,1,0,0


In [147]:
data = pd.concat([df,encode],axis=1)
data.head(3)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,...,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,36.0,FEMALE,0.080353,0.42638,0.54715,0.273680,III,Infiltrating Ductal Carcinoma,Negative,Modified Radical Mastectomy,...,0,0,1,0,0,0,1,0,0,0
1,43.0,FEMALE,-0.420320,0.57807,0.61447,-0.031505,II,Mucinous Carcinoma,Negative,Lumpectomy,...,0,1,0,0,1,0,0,0,0,1
2,69.0,FEMALE,0.213980,1.31140,-0.32747,-0.234260,III,Infiltrating Ductal Carcinoma,Negative,Other,...,0,0,1,0,0,0,0,1,0,0


In [148]:
data.select_dtypes(include='object').columns

Index(['Gender', 'Tumour_Stage', 'Histology', 'HER2 status', 'Surgery_type',
       'Patient_Status'],
      dtype='object')

In [149]:
data.drop(data.select_dtypes(include='object').columns,axis=1,inplace=True)

In [150]:
data.head()

,Age,Protein1,Protein2,Protein3,Protein4,Gender_MALE,Tumour_Stage_II,Tumour_Stage_III,Histology_Infiltrating Lobular Carcinoma,Histology_Mucinous Carcinoma,HER2 status_Positive,Surgery_type_Modified Radical Mastectomy,Surgery_type_Other,Surgery_type_Simple Mastectomy,Patient_Status_Dead
0,36.0,0.080353,0.42638,0.54715,0.273680,0,0,1,0,0,0,1,0,0,0
1,43.0,-0.420320,0.57807,0.61447,-0.031505,0,1,0,0,1,0,0,0,0,1
2,69.0,0.213980,1.31140,-0.32747,-0.234260,0,0,1,0,0,0,0,1,0,0
3,56.0,0.345090,-0.21147,-0.19304,0.124270,0,1,0,0,0,0,1,0,0,0
4,56.0,0.221550,1.90680,0.52045,-0.311990,0,1,0,0,0,0,0,1,0,1


In [151]:
data.shape

(317, 15)

In [152]:
X = data.iloc[:,:-1]
Y = data['Patient_Status_Dead']

In [153]:
X.shape

(317, 14)

In [154]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler()
std.fit(X[['Age','Protein1','Protein2','Protein3','Protein4']])
X_trans = pd.DataFrame(std.fit_transform(X[['Age','Protein1','Protein2','Protein3','Protein4']]),columns= ['Age','Protein1','Protein2','Protein3','Protein4'])


In [155]:
X_trans

,Age,Protein1,Protein2,Protein3,Protein4
0,-1.774446,0.198131,-0.578273,1.092087,0.427163
1,-1.227875,-0.723920,-0.410609,1.206558,-0.061151
2,0.802245,0.444222,0.399950,-0.395115,-0.385571
3,-0.212815,0.685677,-1.283296,-0.166530,0.188098
4,-0.212815,0.458163,1.058052,1.046687,-0.509944
...,...,...,...,...,...
312,-1.774446,0.477040,-0.366429,-0.786751,-0.838532
313,-1.149794,1.399546,0.179219,-0.296577,-0.578634
314,0.177592,-1.274842,1.767330,-0.093753,0.532767
315,1.583060,0.933026,1.222854,-0.741809,-0.312320


In [156]:
xx = X.iloc[:,5:]

In [157]:
xx.shape

(317, 9)

In [158]:
X_trans.shape

(317, 5)

In [159]:
X_final = pd.concat([X_trans.reset_index(drop=True),
                     xx.reset_index(drop=True)], axis=1)


In [160]:
X_final.shape

(317, 14)

In [161]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X_final,Y,test_size=0.2,random_state=42)

In [162]:
def model(ml):
    Model = ml
    Model.fit(x_train,y_train)
    Model.score(x_train,y_train)
    Model.score(x_test,y_test)
    print(Model.score(x_train,y_train),
    Model.score(x_test,y_test))
    

In [163]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
dfir = { 'criterion': ["gini", "entropy", "log_loss"],
    'splitter':['best', 'random'],
    'max_depth':[i for i in range(2,20)]}

In [164]:
from sklearn.tree import DecisionTreeClassifier

In [165]:
gd =GridSearchCV(DecisionTreeClassifier(),param_grid=dfir)
gd.fit(x_train,y_train)

GridSearchCV(estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19],
                         'splitter': ['best', 'random']})

In [166]:
gd.best_params_

{'criterion': 'gini', 'max_depth': 2, 'splitter': 'random'}

In [167]:
gd.best_score_

0.8023529411764706

In [168]:
model(DecisionTreeClassifier(criterion='gini',max_depth = 2, splitter = 'best'))

0.8063241106719368 0.796875


In [169]:
model(DecisionTreeClassifier(criterion='log_loss',max_depth = 4, splitter = 'random'))

0.8142292490118577 0.828125


In [170]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
lis = {'criterion':['gini','entropy'], 
        'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth':[i for i in range(2,20)]}

In [171]:
from sklearn.ensemble import ExtraTreesClassifier

In [172]:
fg = GridSearchCV(ExtraTreesClassifier(),param_grid= lis)
fg.fit(x_train,y_train)

GridSearchCV(estimator=ExtraTreesClassifier(),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                                       14, 15, 16, 17, 18, 19],
                         'max_features': ['auto', 'sqrt', 'log2']})

In [173]:
fg.best_params_

{'criterion': 'gini', 'max_depth': 2, 'max_features': 'sqrt'}

In [174]:
fg.best_score_

0.7984313725490196

In [175]:

model(ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='sqrt',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     monotonic_cst=None, n_estimators=100, n_jobs=-1,
                     oob_score=False, random_state=5375, verbose=0,
                     warm_start=False))

1.0 0.8125


In [176]:
model(ExtraTreesClassifier(criterion = 'gini',max_depth = 2,max_features = 'sqrt'))

0.7984189723320159 0.828125
